In [13]:
import pandas as pd
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
import csv

In [14]:
path_data = "..\\data\\NOMAL.xlsx"
raw_df = pd.read_excel(path_data)

In [15]:
# Remove features with constant values
noInfo_columns = raw_df.columns[raw_df.nunique() <= 1] # columns with no values or with only the same value

filtered_df = raw_df.loc[:, raw_df.nunique() > 1] # this operation also removes the empty columns


In [16]:
import re

# Function to remove special characters from a string
def remove_special_characters(string):
    return re.sub(r'\W+', '', string)

# Rename the features in filtered_df
filtered_df.rename(columns=lambda x: remove_special_characters(x), inplace=True)


c:\Users\balth\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [17]:
# Defining the categories for data

patient_general = ["Codice identificativo:", 'Sesso', 'Nazionalità', 'Età', 'Domicilio','Altezza','Peso', 'Sub-Saharan Africa', 'Pregressa malaria', 'Profilassi']
comorbidities = ['Comorbilità', 'Diabete', 'HIV', 'Cirrosi', 'IRC']
patient_state_arrival = ['GCS', 'Seizures', 'Prostrazione', 'Shock', 'Bleeding', 'ARDS', 'Anemia', 'Creatinina', 'Glicemia', 'Acidosi', 'Bilirubina', 'Hyperparasitaemia', 'Numero criteri']
diagnosis_type = ['RDT', 'Emoscopia', 'NAAT']
diagnosis_result = ['Falciparum', 'Ovale', 'Parassitemia valore assoluto', 'Percentuale parassitemia']
baseline_other = ['PA sistolica', 'PA diastolica', 'FC', 'FR', 'Temperatura', 'Ritardo terapeutico']
QTc = ['QTc ingresso', 'QTc dopo ACT', 'QTC dopo ultima somministrazione artesunato *']
T0 =  ["T0. [GB (in cell/ul):]", "T0. [GR (in cell/ul):]", "T0. [Hb (in g/dl):]", "T0. [PLT (in cell/ul):]", "T0. [Glicemia (in mg/dl):]", "T0. [Azotemia (in mg/dl):]", "T0. [Creatinina (in mg/dl):]", "T0. [LDH (in U/L):]", "T0. [AST (in U/L):]", "T0. [ALT (in U/L):]", "T0. [Bilirubina tot (in mg/dl):]", "T0. [Bilirubina diretta (in mg/dl):]", "T0. [Sodio (in mEq/l):]", "T0. [Potassio (in mEq/l):]", "T0. [Ca (in mg/dl):]", "T0. [INR:]", "T0. [fibrinogeno (in mg/dl):]", "T0. [pH:]", "T0. [bicarbonati (in mmol/l):]", "T0. [Lattati # (in mmol/l):"]
T1 = ["T1. [Goccia spessa e striscio periferico:]", "T1. [TC (temperatura corporea) in °C:]"]
T2 = ["T2. [Goccia spessa e striscio periferico:]", "T2. [TC (temperatura corporea) in °C:]"]
T3 = ["T3. [Goccia spessa e striscio periferico:]", "T3. [TC (temperatura corporea) in °C:]"]
T7 = [ "T7. [Goccia spessa e striscio periferico:]","T7. [TC (temperatura corporea) in °C:]"]
treatment = ["Artesunato ev: Somministrazione [1][Data inizio e ora]", "Artemether/ Lumefantrina: Somministrazione [1][Data inizio e ora]", "Diidroartemisinina/Piperachina: Somministrazione [1][Data inizio e ora]", "ACT", "Artesunato + ACT", "Atovaquone/Proguanile", "Doxiciclina per os", "Clindamicina", "Antibiotici", "Chinino", "Primachina", "Durata Artesunato"]
outcome = ["Durata ricovero", "Decesso.", "ICU", "Eventuali sequele:", "PADH, post-artesunate delayed haemoly1s", "Insorgenza PADH", "Permanenza in Terapia Intensiva (giorni):", "Trasferimento in Rianimazione (anche in altro centro)?", "Guarigione"]
PADH_info = ["Insorgenza PADH", "Si prega di fornire tutti i parameri vitali. [GB (in cell/ul):]", "Si prega di fornire tutti i parameri vitali. [Hb (in g/dl):]", "Si prega di fornire tutti i parameri vitali. [PLT (in cell/ul):]", "Si prega di fornire tutti i parameri vitali. [reticoliti (in cell/ul):]", "Si prega di fornire tutti i parameri vitali. [LDH (in U/l)lcio:]", "Si prega di fornire tutti i parameri vitali. [AST (U/L):]", "Si prega di fornire tutti i parameri vitali. [ALT (U/L):]", "Si prega di fornire tutti i parameri vitali. [Bilirubina tot (mg/dl):]", "Si prega di fornire tutti i parameri vitali. [Bilirubina diretta (mg/dl):]", "Si prega di fornire tutti i parameri vitali. [aptoglobina (in mg/dl):]", "Si prega di fornire tutti i parameri vitali. [test di Coombs diretto:]", "Si prega di fornire tutti i parameri vitali. [test di coombs indiretto:]", "Vuole riportare ulteriori informazioni relative ai prelievi effettuati durante le visite intermedie fino alla risoluzione dell’emolisi?", "Nadir Hb", "Trasfusione:", "Unità trasfuse"]
follow_up = ["Altri eventi avversi", "Diarrea e disidratazione", "Ipertransaminasemia tardiva", "Polmonite", "IVU nosocomiale", "Esofagite", "Dispepsia", "Tachiaritmia sopraventricolare"]

In [18]:
# Removing special caracters from the category description

import re

# Function to remove special characters from a string
def remove_special_characters(string):
    return re.sub(r'\W+', '', string)

# Update the list of features for each category
patient_general = [remove_special_characters(feature) for feature in patient_general]
comorbidities = [remove_special_characters(feature) for feature in comorbidities]
patient_state_arrival = [remove_special_characters(feature) for feature in patient_state_arrival]
diagnosis_type = [remove_special_characters(feature) for feature in diagnosis_type]
diagnosis_result = [remove_special_characters(feature) for feature in diagnosis_result]
baseline_other = [remove_special_characters(feature) for feature in baseline_other]
QTc = [remove_special_characters(feature) for feature in QTc]
T0 = [remove_special_characters(feature) for feature in T0]
T1 = [remove_special_characters(feature) for feature in T1]
T2 = [remove_special_characters(feature) for feature in T2]
T3 = [remove_special_characters(feature) for feature in T3]
T7 = [remove_special_characters(feature) for feature in T7]
treatment = [remove_special_characters(feature) for feature in treatment]
outcome = [remove_special_characters(feature) for feature in outcome]
PADH_info = [remove_special_characters(feature) for feature in PADH_info]
follow_up = [remove_special_characters(feature) for feature in follow_up]

In [19]:
import re

# Define the categories
categories = [patient_general, comorbidities, patient_state_arrival, diagnosis_type, diagnosis_result, baseline_other, QTc, T0, T1, T2, T3, T7, treatment, outcome, PADH_info, follow_up]

# Check if features belong to a category (partial match)
missing_features = []
for column in filtered_df.columns:
    matched = False
    for category in categories:
        for feature in category:
            if re.search(re.escape(feature), column, re.IGNORECASE):
                matched = True
                break
        if matched:
            break
    if not matched:
        missing_features.append(column)

# Print the missing features
if missing_features:
    print("Features not belonging to any category:")
    for feature in missing_features:
        print(feature)
else:
    print("All features belong to a category.")


All features belong to a category.


In [20]:

# Check if noInfo_columns are present in categories and remove them
removed_features = []
for column in noInfo_columns:
    for i, category in enumerate(categories):
        if column in category:
            categories[i].remove(column)
            removed_features.append(column)
            break

# Print the removed features
if removed_features:
    print("Removed features from categories:")
    for feature in removed_features:
        print(feature)
else:
    print("No features were removed from categories.")


Removed features from categories:
Cirrosi
Seizures
RDT


In [21]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

# Define the category lists
categories = [patient_general, comorbidities, patient_state_arrival, diagnosis_type, diagnosis_result, baseline_other, QTc, T0, T1, T2, T3, T7, treatment, outcome, PADH_info, follow_up]
category_names = ['Patient General', 'Comorbidities', 'Patient State Arrival', 'Diagnosis Type', 'Diagnosis Result', 'Baseline Other', 'QTc', 'T0', 'T1', 'T2', 'T3', 'T7', 'Treatment', 'Outcome', 'PADH Info', 'Follow-up']

# Get the 'ICU' column from filtered_df
target_column = filtered_df['ICU']

# Iterate over each category
for category, category_name in zip(categories, category_names):
    # Filter the features based on the current category
    features = [feature for feature in filtered_df.columns if feature in category]

    # Calculate the correlation between each feature and ICU
    correlations = []
    for feature in features:
        if filtered_df[feature].dtype != np.object:  # Check if the feature is not a string
            correlation = filtered_df[feature].corr(target_column)
            correlations.append((feature, correlation))

    # Sort the correlation values in ascending order
    correlations.sort(key=lambda x: x[1])

    # # Prepare colors for the bars based on correlation values
    # cmap = colors.LinearSegmentedColormap.from_list('correlation_cmap', ['green', 'white', 'red'])
    # norm = colors.Normalize(vmin=-1, vmax=1)

    # # Plot the correlation values on a bar chart with colored bars
    # plt.figure(figsize=(10, 6))
    # features, correlation_values = zip(*correlations)
    # bar_colors = cmap(norm(correlation_values))
    # plt.bar(features, correlation_values, color=bar_colors)
    # plt.xlabel('Features')
    # plt.ylabel('Correlation with ICU')
    # plt.title(f'Correlation between Features in {category_name} and ICU')
    # plt.xticks(rotation=90)
    # plt.gca().set_xticklabels([label[:10] for label in features])  # Display only the first 10 characters of the labels
    # plt.tight_layout()
    # plt.show()


C:\Users\balth\AppData\Local\Temp/ipykernel_14800/2938341224.py:20: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if filtered_df[feature].dtype != np.object:  # Check if the feature is not a string


In [22]:
import numpy as np

# Create a new dataframe to store the results
feature_info = pd.DataFrame(columns=['feature', 'type', 'mean', 'variance'])

# Iterate over the columns in the filtered dataframe
for column in filtered_df.columns:
    col_data = filtered_df[column]
    col_type = ''
    col_mean = ''
    col_var_entropy = ''
    
    # Check if the column has string values
    if col_data.dtype == object:
        col_type = 'string'
    elif set(col_data.dropna().unique()) == {0, 1}:
        col_type = 'categorical'
        col_mean = col_data.mean()
        # col_var_entropy = np.nans
    elif col_data.dtype == np.int64 or all(pd.isnull(val) or val.is_integer() for val in col_data.dropna().unique()):
        col_type = 'int'
        col_mean = col_data.mean()
        col_var_entropy = col_data.var()
    elif col_data.dtype == np.float64 or any('.' in str(val) for val in col_data.dropna().unique()):
        col_type = 'float'
        col_mean = col_data.mean()
        col_var_entropy = col_data.var()
    else:
        col_type = 'unknown'

    # Add the results to the new dataframe
    feature_info = feature_info.append({'feature': column, 'type': col_type, 'mean': col_mean, 'variance': col_var_entropy},
                                       ignore_index=True)

# Print the resulting dataframe
print(feature_info)


                           feature         type       mean    variance
0             Codiceidentificativo       string                       
1                            Sesso  categorical   0.652778            
2                      Nazionalità  categorical   0.569444            
3                              Età          int  50.819444  231.530321
4                        Domicilio  categorical   0.045455            
..                             ...          ...        ...         ...
106                      Polmonite  categorical   0.051724            
107                 IVUnosocomiale  categorical   0.034483            
108                      Esofagite  categorical   0.017241            
109                      Dispepsia  categorical   0.017241            
110  Tachiaritmiasopraventricolare  categorical   0.034483            

[111 rows x 4 columns]


In [23]:
feature_info.head(5)

,feature,type,mean,variance
0,Codiceidentificativo,string,,
1,Sesso,categorical,0.652778,
2,Nazionalità,categorical,0.569444,
3,Età,int,50.819444,231.530321
4,Domicilio,categorical,0.045455,


In [24]:
# Step 1: Export the DataFrame to a CSV file
file_name = "../data/feature_info.csv"
feature_info.to_csv(file_name, index=False)  # Set index=False to exclude the index column in the CSV file